In [1]:
import plotly.express as px
import plotly as py
import plotly.graph_objs as go
import pickle
import networkx as nx 
import numpy as np 
from numpy.linalg import norm
from mayavi import mlab
import cv2
from ThreeDdraw import draw_plotly
%gui qt

In [28]:
### FRUCHTERMAN & REINGOLD ###

# HELPER FUNCTIONS : 
def optimal_dist(C,N):
    area = 1
    C = .2
    return C * (area/N)**(1/3)

def dist(a,b):
    """ a and b are both points in 3 d space np.array([x,y,z])
    """
    return numpy.linalg.norm(a-b)

def f_attract(distance,optimal_dist):
    return distance**2/optimal_dist

def f_repulse(distance,optimal_dist):
    C = .2
    return  C * (optimal_dist**2) / distance # maybe a minus ..

# MAIN ALGORITHM : 

def fruchterman_reingold(g,num_iters):
    """ g = networkx graph 
        assume network to be drawn in 3d unit cube
    """
    C = .2
    opt_dist = optimal_dist(C,g.number_of_nodes())

    #opt_dist = np.array([optim_dist] * 3)
    temp = 1/10
    # assign inintial random possitions :
    for _,n in g.nodes(data = True):
        n['pos'] = np.random.rand(3)

    # main loop:
    for i in range(num_iters):
        # calculate repulsive forces : 
        for i1,n1 in g.nodes(data = True):
            n1['disp'] = 0
            for i2,n2 in g.nodes(data = True):
                if i1 != i2:
                    d = n1['pos'] - n2['pos']
                    #n1['disp'] = f_repulse(d,opt_dist)
                    n1['disp'] = n1['disp'] + (d/norm(d)) * f_repulse(norm(d),opt_dist)

        # calculate attractive forces between edges : 
        for E in g.edges(data = True):
            n1,n2 = g.nodes[E[0]],g.nodes[E[1]]
            d = n1['pos'] - n2['pos']
            #n1['disp'] = f_attract(d,opt_dist)
            #n2['disp'] = f_attract(norm(d),opt_dist)
            n1['disp'] = n1['disp'] - (d/norm(d)) * f_attract(norm(d),opt_dist)
            n2['disp'] = n2['disp'] + (d/norm(d)) * f_attract(norm(d),opt_dist)
        
        # limit displacement with temperature
        for _,n in g.nodes(data = True):
            final_displacement = (n['disp']/norm(n['disp'])) * min([norm(n['disp']),temp])
            # check if optimal distance is already achieved :
            if np.isnan(final_displacement).any():
                pass
            else:
                n['pos'] = n['pos'] + final_displacement
            # check if position is not out of bounds : 
            for j in range(3):
                if n['pos'][j] > 1.:
                    n['pos'][j] = 1.
                elif n['pos'][j] < 0.:
                    n['pos'][j] = 0.
                elif np.nan in n['pos']:
                     n['pos'] = np.random.rand(3)

        temp += - 1/(10*num_iters)
        #print(temp)
        print(i)
    return g

def normalize_postions(positions):
    positions = np.array(positions)
    new = []
    for axes in range(3):
        l = positions[:,axes]
        new_axis = []
        for coord in l:
            x = (coord - min(l))/(max(l) - min(l))
            new_axis.append(x)
        new.append(new_axis)
    new = np.array(new)
    return new.T

### TEST ###
# 2d : 

def test_2d():
    #g = nx.erdos_renyi_graph(170,.3)
    g = nx.random_geometric_graph(176,.18)
    g = fruchterman_reingold(g,50)
    # convert positions to 2d : 
    positions = {}
    print(len(list(g.edges)))
    print(nx.is_connected(g))
    for n,data in g.nodes(data = True):
        positions[n] = data['pos'][:2]
        #positions.append(n['pos'][:2])

    nx.draw(g,positions)
    print(positions)
    plt.show()


def test_3d():
    #g = nx.random_geometric_graph(200,.24, dim = 3)
    g = nx.erdos_renyi_graph(200,.05)
    print(g.number_of_edges())
    print(nx.is_connected(g))
    #while not nx.is_connected(g):
      #g = nx.random_geometric_graph(200,.25, dim = 3)
    #g = nx.erdos_renyi_graph(170,.15)
    print(nx.is_connected(g))
    matrix = nx.to_numpy_matrix(g)
    g = fruchterman_reingold(g,100)
    positions = []
    for n,data in g.nodes(data = True):
        positions.append(data['pos'])
    positions = normalize_postions(positions)
    draw_plotly(positions,matrix)
    return g,positions


In [29]:
g,positions = test_3d()

966
True
True
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
966
2898


In [27]:
positions = np.array(positions)


new_positions = normalize_postions(positions)
print(positions)
print(new_positions)

[[0.49889134 0.55881684 0.49229501]
 [0.50710267 0.60265648 0.53683438]
 [0.54681426 0.54912013 0.475521  ]
 [0.5251673  0.58980424 0.44647658]
 [0.54975699 0.56584747 0.51786827]
 [0.52606906 0.58431163 0.49247274]
 [0.49954863 0.57606948 0.55881452]
 [0.53313578 0.55690353 0.5332431 ]
 [0.47985449 0.54026827 0.4691986 ]
 [0.47915886 0.5857345  0.45079367]
 [0.51469066 0.5489529  0.50464287]
 [0.61511598 0.53120224 0.47800508]
 [0.51719758 0.5134427  0.52135856]
 [0.5037816  0.56377022 0.54834202]
 [0.55867113 0.51097893 0.51868247]
 [0.48439676 0.53113921 0.50545289]
 [0.47856643 0.57219356 0.53085366]
 [0.52224976 0.5946484  0.46903702]
 [0.49359617 0.51731043 0.51646324]
 [0.50030636 0.58488335 0.51758617]
 [0.46558559 0.56072507 0.53854786]
 [0.5264619  0.5075847  0.53670175]
 [0.47983777 0.62058506 0.50836328]
 [0.51777561 0.51336808 0.50522048]
 [0.60679268 0.59920764 0.46900155]
 [0.56350989 0.52432379 0.48117707]
 [0.50107813 0.48403972 0.48167232]
 [0.51233454 0.57939979 0.51

[0, 1, 2]

In [17]:
def test_3BA():
    #g = nx.random_geometric_graph(200,.24, dim = 3)
    g = nx.barabasi_albert_graph(200,4)
    print(g.number_of_edges())
    print(nx.is_connected(g))
    #while not nx.is_connected(g):
      #g = nx.random_geometric_graph(200,.25, dim = 3)
    #g = nx.erdos_renyi_graph(170,.15)
    print(nx.is_connected(g))
    matrix = nx.to_numpy_matrix(g)
    g = fruchterman_reingold(g,100)
    positions = []
    for n,data in g.nodes(data = True):
        positions.append(data['pos'])
    
    draw_plotly(positions,matrix)
    return g,positions

test_3BA()

784
True
True
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
784
2352


(<networkx.classes.graph.Graph at 0x7fe3d5d00450>,
 [array([0.52978296, 0.54762285, 0.51689567]),
  array([0.53048272, 0.54007481, 0.55270457]),
  array([0.5583241 , 0.57442805, 0.53910605]),
  array([0.5860904 , 0.57834605, 0.46364892]),
  array([0.55401372, 0.54469297, 0.50694857]),
  array([0.53100068, 0.52705127, 0.51347388]),
  array([0.51658215, 0.52155039, 0.5401428 ]),
  array([0.5158527 , 0.54036608, 0.50968409]),
  array([0.52938173, 0.51970459, 0.53157648]),
  array([0.51266432, 0.5368601 , 0.54692916]),
  array([0.52641889, 0.53230719, 0.49062429]),
  array([0.56568657, 0.54012654, 0.51522767]),
  array([0.48416865, 0.56233148, 0.50243173]),
  array([0.53014855, 0.53159409, 0.52005628]),
  array([0.53497534, 0.56315708, 0.5076822 ]),
  array([0.51730475, 0.56177385, 0.52701284]),
  array([0.53729154, 0.55799781, 0.54966554]),
  array([0.51502874, 0.58170076, 0.51645705]),
  array([0.52978349, 0.57996908, 0.53252144]),
  array([0.5664706 , 0.54767687, 0.54601634]),
  array([

In [79]:
g = nx.random_geometric_graph(200,.24, dim = 3)
matrix = nx.to_numpy_matrix(g)
positions = []
for n,data in g.nodes(data = True):
    positions.append(data['pos'])
draw_plotly(positions,matrix)
# print(nx.is_connected(g))
# print([n[1]['pos'][2] for n in g.nodes(data = True)])
# print([x[2] for x in positions])

866
2598


In [86]:
def nodesInCube(g,positions,dim):
    # input is 3d list of [x,y,z] postitions
    xyz = np.array(positions)
    positions = xyz * dim
    positions = positions.astype(int)
    # replace node data:
    for n in g.nodes(data = True):
        n[1]['pos'] = positions[n[0]]
    return g,positions

def draw3Dlines(g):
    cube_draw = np.zeros((256,256,256))
    for n1,n2 in g.edges():
        x1,y1,z1 = g.nodes[n1]['pos']
        x2,y2,z2 = g.nodes[n2]['pos']
        print('source',g.nodes[n1]['pos'])
        print('target',g.nodes[n2]['pos'])
        xy = np.zeros((256,256))
        cv2.line(xy, (x1,y1), (x2,y2), 1, thickness=2)
        xz = np.zeros((256,256))
        cv2.line(xz, (x1,z1), (x2,z2), 1, thickness=2)
        cube = np.zeros((256,256,256))
        cube[xy==1] = 1
        cube = np.swapaxes(cube, 1, 2)
        cube[xz!=1] = 0
        cube = np.swapaxes(cube, 1, 2)
        cube_draw[cube == 1] = 1
    return cube_draw
    

In [87]:
g,binned_positions = nodesInCube(g,positions,256)
#print(binned_positions)
cube = draw3Dlines(g)
#draw3Dlines(g)

source [84 89 80]
target [76 96 82]
source [84 89 80]
target [ 47 119  42]
source [84 89 80]
target [133 106  97]
source [84 89 80]
target [ 42 126  94]
source [84 89 80]
target [ 74  46 120]
source [84 89 80]
target [ 96 134  98]
source [100 203  26]
target [ 80 211  73]
source [100 203  26]
target [146 230   9]
source [100 203  26]
target [ 77 193  37]
source [100 203  26]
target [ 45 219   8]
source [100 203  26]
target [108 198  42]
source [100 203  26]
target [113 252  53]
source [100 203  26]
target [130 235  21]
source [100 203  26]
target [123 170  14]
source [100 203  26]
target [157 192  23]
source [100 203  26]
target [ 84 232  52]
source [100 203  26]
target [ 83 194  23]
source [100 203  26]
target [ 85 245  52]
source [ 27  27 176]
target [ 47  70 149]
source [ 27  27 176]
target [ 52  66 178]
source [ 27  27 176]
target [ 27  42 234]
source [ 27  27 176]
target [ 36  20 232]
source [ 27  27 176]
target [ 27  33 147]
source [ 27  27 176]
target [ 55  30 188]
source [ 27  

source [143 167 211]
target [146 158 238]
source [143 167 211]
target [103 131 223]
source [143 167 211]
target [194 165 216]
source [ 36 167  54]
target [ 55 203  74]
source [ 36 167  54]
target [ 36 160  16]
source [ 36 167  54]
target [ 22 130  97]
source [ 36 167  54]
target [ 42 126  94]
source [ 36 167  54]
target [  2 157 101]
source [ 36 167  54]
target [ 77 193  37]
source [ 36 167  54]
target [ 56 185  98]
source [ 36 167  54]
target [ 11 204  23]
source [ 36 167  54]
target [ 47 119  42]
source [215 164 182]
target [240 171 179]
source [215 164 182]
target [194 165 216]
source [215 164 182]
target [194 158 161]
source [215 164 182]
target [161 138 190]
source [215 164 182]
target [202 181 145]
source [215 164 182]
target [253 180 180]
source [215 164 182]
target [228 132 227]
source [215 164 182]
target [217 174 189]
source [133 106  97]
target [ 96 134  98]
source [133 106  97]
target [127 134  83]
source [133 106  97]
target [76 96 82]
source [133 106  97]
target [162 131 

source [  2 157 101]
target [ 33 185 136]
source [  2 157 101]
target [ 56 185  98]
source [  2 157 101]
target [ 22 130  97]
source [205  20 187]
target [164  39 200]
source [205  20 187]
target [185  36 159]
source [205  20 187]
target [192  21 198]
source [205  20 187]
target [205   9 228]
source [205  20 187]
target [180  43 229]
source [205  20 187]
target [160  21 162]
source [115  19  20]
target [92  3 19]
source [115  19  20]
target [114   6  17]
source [115  19  20]
target [97  2 66]
source [115  19  20]
target [128  71   6]
source [239  65   4]
target [193  81  25]
source [239  65   4]
target [231  85  51]
source [239  65   4]
target [239  68  46]
source [239  65   4]
target [216  14  10]
source [128  71   6]
target [163  92  44]
source [128  71   6]
target [166  99  27]
source [ 80 211  73]
target [ 38 230  93]
source [ 80 211  73]
target [ 70 246 106]
source [ 80 211  73]
target [ 84 232  52]
source [ 80 211  73]
target [ 79 226 102]
source [ 80 211  73]
target [ 56 185  98

source [126 169 136]
target [ 91 144 156]
source [ 24 172 183]
target [ 33 185 136]
source [ 24 172 183]
target [ 29 201 226]
source [ 24 172 183]
target [ 46 188 153]
source [ 24 172 183]
target [ 79 170 210]
source [ 24 172 183]
target [ 28 152 224]
source [ 24 172 183]
target [ 20 208 231]
source [163  92  44]
target [166  99  27]
source [163  92  44]
target [186  88  78]
source [163  92  44]
target [150 147  59]
source [163  92  44]
target [172 118  61]
source [171 251 182]
target [157 224 204]
source [171 251 182]
target [170 218 145]
source [171 251 182]
target [175 239 156]
source [100 177 149]
target [ 46 188 153]
source [100 177 149]
target [ 92 176 116]
source [100 177 149]
target [111 167 194]
source [100 177 149]
target [ 69 162 140]
source [100 177 149]
target [ 85 179  96]
source [100 177 149]
target [ 94 197 180]
source [100 177 149]
target [ 91 144 156]
source [76 96 82]
target [26 64 80]
source [76 96 82]
target [ 47 119  42]
source [76 96 82]
target [26 73 64]
source 

source [236 156  58]
target [239 119  88]
source [236 156  58]
target [219 158  51]
source [ 50  76 232]
target [ 92 114 255]
source [ 50  76 232]
target [ 27  42 234]
source [ 50  76 232]
target [ 52  66 178]
source [208 220  44]
target [157 192  23]
source [103 131 223]
target [111 167 194]
source [103 131 223]
target [ 79 170 210]
source [103 131 223]
target [146 158 238]
source [103 131 223]
target [ 92 114 255]
source [114   6  17]
target [92  3 19]
source [ 78  98 141]
target [ 91 144 156]
source [ 78  98 141]
target [ 68 109 153]
source [ 78  98 141]
target [ 52  66 178]
source [105 153  70]
target [ 77 193  37]
source [105 153  70]
target [ 92 176 116]
source [203  12  92]
target [197  30  55]
source [203  12  92]
target [222  46 113]
source [162 131 138]
target [146  97 147]
source [162 131 138]
target [159  86 177]
source [162 131 138]
target [138  92 179]
source [162 131 138]
target [194 158 161]
source [159  86 177]
target [164  39 200]
source [159  86 177]
target [146  97 

In [83]:
mlab.clf()
#mlab.volume_slice(cube)
mlab.contour3d(cube)

In [55]:
for n1,n2 in g.edges():
    x1,y1,z1 = g.nodes[n1]['pos']
    x2,y2,z2 = g.nodes[n2]['pos']
    print(x1,y1,z1,x2,y2,z2)

129 154 151 140 154 151
129 154 151 142 149 150
129 154 151 107 150 152
129 154 151 141 161 146
129 154 151 134 151 144
129 154 151 120 143 149
129 154 151 138 153 153
129 154 151 113 156 150
129 154 151 133 147 151
129 154 151 119 151 154
129 154 151 131 146 150
137 125 132 127 121 125
137 125 132 133 134 136
137 125 132 130 135 141
137 125 132 134 132 128
137 125 132 142 124 128
137 125 132 125 138 137
137 125 132 149 120 126
137 125 132 134 133 132
137 125 132 128 140 139
137 125 132 133 134 138
137 125 132 138 111 127
137 125 132 135 108 126
137 125 132 138 134 137
137 125 132 145 111 127
137 125 132 135 124 120
137 125 132 147 119 127
143 157 158 135 150 161
143 157 158 134 160 167
143 157 158 140 154 151
143 157 158 152 150 149
143 157 158 138 153 153
143 157 158 142 149 150
143 157 158 144 151 150
143 157 158 145 165 168
143 157 158 141 161 146
143 157 158 133 152 160
143 157 158 141 159 165
145 158 149 151 168 165
145 158 149 136 146 142
145 158 149 125 158 150
145 158 149 149 

153 131 120 146 143 115
153 131 120 150 136 104
131 147 146 134 134 142
131 147 146 135 141 134
131 147 146 129 153 137
131 147 146 135 141 150
131 147 146 133 140 144
131 147 146 132 142 143
98 101 119 101 102 121
98 101 119 102 109 130
98 101 119 97 108 126
98 101 119 95 100 122
98 101 119 98 107 127
98 101 119 93 95 111
98 101 119 96 94 108
98 101 119 107 105 127
98 101 119 108 108 126
150 136 104 147 131 119
150 136 104 153 129 105
150 136 104 151 140 87
150 136 104 146 143 115
150 136 104 144 137 113
150 136 104 149 139 99
98 154 147 109 153 142
98 154 147 96 159 147
98 154 147 105 142 137
98 154 147 102 150 153
98 154 147 96 162 142
98 154 147 98 155 153
98 154 147 95 143 149
98 154 147 107 150 152
98 154 147 92 150 156
140 160 159 136 146 142
140 160 159 151 168 165
151 98 124 140 103 124
151 98 124 159 92 121
151 98 124 153 93 122
151 98 124 155 112 123
151 98 124 149 105 126
151 98 124 158 97 124
151 98 124 162 93 116
151 98 124 151 101 131
151 98 124 144 102 124
151 98 124 16

127 121 125 117 115 123
127 121 125 122 115 130
127 121 125 135 108 126
128 109 134 140 103 135
128 109 134 111 132 132
128 109 134 138 103 130
128 109 134 144 95 129
128 109 134 133 99 129
133 147 151 131 146 150
133 147 151 140 154 151
133 147 151 126 139 146
133 147 151 122 130 142
133 147 151 134 151 144
112 125 140 122 115 130
112 125 140 101 119 139
112 125 140 105 114 134
112 125 140 98 123 142
112 125 140 122 130 142
112 125 140 126 139 146
112 125 140 106 123 131
112 125 140 122 136 145
140 103 135 135 108 126
140 103 135 138 103 130
140 103 135 133 99 129
136 131 117 135 124 120
136 131 117 134 133 132
136 131 117 127 121 114
116 119 118 117 115 123
116 119 118 116 114 124
116 119 118 106 116 120
116 119 118 111 121 112
116 119 118 106 123 131
116 119 118 127 121 114
116 119 118 129 112 121
108 149 157 102 150 153
140 103 124 135 108 126
140 103 124 125 108 123
140 103 124 138 111 127
140 103 124 138 103 130
140 103 124 145 111 127
140 103 124 129 112 121
140 103 124 133 99 1

In [7]:
xy = np.zeros((256,256))
cv2.line(xy, (200,200), (100,100), 1, thickness=2)


xz = np.zeros((256,256))
cv2.line(xz, (200,150), (100,150), 1, thickness=2)



cube = np.zeros((256,256,256))


cube[xy==1] = 1
cube = np.swapaxes(cube, 1, 2)
cube[xz!=1] = 0
cube = np.swapaxes(cube, 1, 2)

In [8]:
mlab.contour3d(cube)